In [8]:
%matplotlib inline
import isce
import os
import glob
import numpy as np
import shutil
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
from uavsar_pytools.georeference import geolocate_uavsar
from pathlib import Path
from glob import glob
from uavsar_pytools.convert.tiff_conversion import read_annotation, array_to_tiff
from os.path import join, basename, dirname
from rasterio.vrt import WarpedVRT
from osgeo import gdal, osr

In [9]:
### still need to hand format unw vrt files....

In [10]:
# combine llh files
def combo_llhs(data_dir: Path, ann_fp: Path):
    """
    Combines segment .llh files into a single combined .llh file and saves as a binary with .vrt.
    """
    assert data_dir.exists()

    re_llhs = {'lat':[], 'lon': [], 'height':[]}
    for llh in sorted(data_dir.glob('*.llh')):
        segment = llh.stem.split('_')[-2].replace('s','')

        data = np.fromfile(llh, np.dtype('<f'))
        lat, lon, height = data[::3], data[1::3], data[2::3]
        for key, da in zip(re_llhs.keys(), [lat, lon, height]):
            re_llhs[key].extend(da)
                               
    full = np.empty(len(re_llhs['lat'])*3, dtype='>f')
    full[0::3] = re_llhs['lat']
    full[1::3] = re_llhs['lon']
    full[2::3] = re_llhs['height']

    # read ann file
    desc = read_annotation(ann_fp)
    print('reading .ann file')

    # read in number of rows from each llh file
    nrows1 = desc[f'llh_1_2x8.set_rows']['value']
    nrows2 = desc[f'llh_2_2x8.set_rows']['value']

    # add rows for new reshaping number
    nrows_new = nrows1 + nrows2
    print('new number of rows = ',nrows_new)
    
    # read in cols, same fo rboth
    ncols = desc[f'llh_1_2x8.set_cols']['value']
    dt = np.dtype('<f')

    # create empty arrays for lat and lon
    lat_array = np.empty((nrows_new, ncols))
    lon_array = np.empty((nrows_new, ncols))
    
    # fill each layer
    lat_array = full[::3].reshape(nrows_new, ncols)
    lon_array = full[1::3].reshape(nrows_new, ncols)

    # define path to bin file
    lat_output_file = data_dir / "multi_seg.lat"
    lon_output_file = data_dir / "multi_seg.lon"

    #### Save the array to bin file
    # lat
    lat_array.tofile(lat_output_file)
    print('.lat saved')
    # lon
    lon_array.tofile(lon_output_file)
    print('.lon saved')

    profile = {
    'driver': 'GTiff',
    'interleave': 'band',
    'tiled': False,
    'nodata': 0,
    'width': ncols,
    'height':nrows_new,
    'count':1,
    'dtype':'float32'
    }
    
    ### Save out tifs to be converted to vrt
    # lat
    with rio.open(join(str(lat_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(lat_array.astype(lat_array.dtype), 1)

    with rio.open(join(str(lon_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(lon_array.astype(lon_array.dtype), 1)

    # Add VRT file for each tif
    tifs = glob(join(data_dir, '*.tif')) # list all .llh files
    for tiff in tifs: # loop to open and translate .llh to .vrt, and save .vrt using gdal
        raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
        raster = gdal.Translate(join(data_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float32)
    raster_dataset = None

    print('new .lat and .lon with .vrt have saved')

In [11]:
def geocodeUsingGdalWarp(infile, latfile, lonfile, outfile,
                         insrs=4326, outsrs=None,
                         spacing=None, fmt='GTiff', bounds=None,
                         method='near'):
    '''
    From: Dr. Gareth Funning, UC Riverside, UNAVCO InSAR Short Course
    Geocode a swath file using corresponding lat, lon files
    '''
    sourcexmltmpl = '''    <SimpleSource>
      <SourceFilename>{0}</SourceFilename>
      <SourceBand>{1}</SourceBand>
    </SimpleSource>'''
    
    driver = gdal.GetDriverByName('VRT')
    tempvrtname = 'temp_ele.vrt'
    inds = gdal.OpenShared(infile, gdal.GA_ReadOnly)
    
    tempds = driver.Create(tempvrtname, inds.RasterXSize, inds.RasterYSize, 0)
    
    for ii in range(inds.RasterCount):
        band = inds.GetRasterBand(1)
        tempds.AddBand(band.DataType)
        tempds.GetRasterBand(ii+1).SetMetadata({'source_0': sourcexmltmpl.format(infile, ii+1)}, 'vrt_sources')
  
    sref = osr.SpatialReference()
    sref.ImportFromEPSG(insrs)
    srswkt = sref.ExportToWkt()
    tempds.SetMetadata({'SRS' : srswkt,
                        'X_DATASET': lonfile,
                        'X_BAND' : '1',
                        'Y_DATASET': latfile,
                        'Y_BAND' : '1',
                        'PIXEL_OFFSET' : '0',
                        'LINE_OFFSET' : '0',
                        'PIXEL_STEP' : '1',
                        'LINE_STEP' : '1'}, 
                        'GEOLOCATION')
    
    band = None
    tempds = None 
    inds = None
    
    print('geocoding...')
    if spacing is None:
        spacing = [None, None]
    warpOptions = gdal.WarpOptions(format=fmt,
                                xRes=spacing[0], yRes=spacing[0],
                                dstSRS=outsrs, outputBounds = bounds, dstNodata = -9999,
                                resampleAlg=method, geoloc=True)
    gdal.Warp(outfile, tempvrtname, options=warpOptions)
    os.remove('temp_ele.vrt')
    print('done!')

In [12]:
# start in downloads directory
os.chdir('/Users/jtarrico/sierra_isce_multi')

# p1

In [9]:
dir1 = Path('./int/new_llh')
ann1 = Path('./int/ann/sierra_17305_20002_001_200131_L090VV_01_BC.ann')

In [10]:
full1 = combo_llhs(data_dir = dir1, ann_fp = ann1)

reading .ann file
new number of rows =  16212
.lat saved
.lon saved
2023-11-29 12:50:05,320 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x1a20114c0>
2023-11-29 12:50:07,002 - rasterio.env - DEBUG - Starting outermost env
2023-11-29 12:50:07,004 - rasterio.env - DEBUG - No GDAL environment exists
2023-11-29 12:50:07,004 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x1a20117c0> created
2023-11-29 12:50:07,008 - rasterio._filepath - DEBUG - Installing FilePath filesystem handler plugin...
2023-11-29 12:50:07,009 - rasterio._env - DEBUG - GDAL_DATA found in environment.
2023-11-29 12:50:07,011 - rasterio._env - DEBUG - PROJ_DATA found in environment.
2023-11-29 12:50:07,012 - rasterio._env - DEBUG - Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x1a20117c0>.
2023-11-29 12:50:07,013 - rasterio.env - DEBUG - Entered env context: <rasterio.env.Env object at 0x1a20114c0>
2023-11-29 12:50:07,014 - rasterio._io - DEBUG - P

/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


2023-11-29 12:50:08,291 - rasterio.env - DEBUG - Exiting env context: <rasterio.env.Env object at 0x1a1e680a0>
2023-11-29 12:50:08,293 - rasterio.env - DEBUG - Cleared existing <rasterio._env.GDALEnv object at 0x1a20117c0> options
2023-11-29 12:50:08,294 - rasterio._env - DEBUG - Stopped GDALEnv <rasterio._env.GDALEnv object at 0x1a20117c0>.
2023-11-29 12:50:08,295 - rasterio.env - DEBUG - Exiting outermost env
2023-11-29 12:50:08,297 - rasterio.env - DEBUG - Exited env context: <rasterio.env.Env object at 0x1a1e680a0>
2023-11-29 12:50:08,298 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x1a1c1eee0>
2023-11-29 12:50:08,299 - rasterio.env - DEBUG - Starting outermost env
2023-11-29 12:50:08,300 - rasterio.env - DEBUG - No GDAL environment exists
2023-11-29 12:50:08,301 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x1a20adcd0> created
2023-11-29 12:50:08,303 - rasterio._env - DEBUG - GDAL_DATA found in environment.
2023-11-2

In [34]:
# path to your unw.vrt
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.vrt'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_unw.tif'

In [35]:
# p1 unw
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


In [13]:
# p1 concomp
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_concomp.tif'

In [14]:
# p1 unw
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...


Warning 1: for band 1, destination nodata value has been clamped to 0, the original value being out of range.


done!


In [ ]:
# p1 coh
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_concomp.tif'